<a href="https://colab.research.google.com/github/mshsu/probasets/blob/main/population_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Population Demo

Inspired by Dr. Jimmy Doi's [RShiny app](http://shiny.calpoly.sh/BenfordData/).

Benford's Law states that in real life numerical data, the leading digits are likely to be small. Do city population estimates from the US Census Bureau follow this law?

## Setup

In [65]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from ipywidgets import interactive
import ipywidgets as widgets

## Data

In [66]:
pop = pd.read_csv("https://mshsu.github.io/probasets/data/Population.csv")[['POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2022',
                                                                            'BIRTHS2020', 'BIRTHS2021', 'BIRTHS2022',
                                                                            'DEATHS2020','DEATHS2021','DEATHS2022']]
pop.head()

,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,BIRTHS2020,BIRTHS2021,BIRTHS2022,DEATHS2020,DEATHS2021,DEATHS2022
0,5031362,5049846,5074296,13867,57112,58280,15165,69056,66870
1,58902,59210,59759,162,694,709,177,682,673
2,233219,239361,246435,560,2367,2411,593,3044,2909
3,24960,24539,24706,60,276,277,93,383,372
4,22183,22370,22005,56,244,248,55,318,321


## Calculating Proportions

In [67]:
def count_first_digit(var):
  obs = (pop[pop[var] > 0][var]
         .astype(str)
         .apply(lambda x: x[0])
         .astype(int)
         .value_counts(normalize=True)
         .sort_index()
         .to_frame()
         .reset_index(names="num"))

  d = pd.Series(range(1, 10))
  bfd = np.log10((d + 1) / d).to_frame().rename(columns={0: "BENFORD'S LAW"})
  bfd['num'] = bfd.index + 1
  bfd[['num', "BENFORD'S LAW"]]

  both = pd.merge(left=obs, right=bfd, on="num")
  display(both)

  print()
  both.plot.bar(x="num")
  plt.show()

  print()
  test_stat, p = stats.chisquare(both[var], both["BENFORD'S LAW"])
  print("Chi-Square Goodness of Fit Test")
  print("Test Statistic:", test_stat)
  print("p-value:", p)  

  return obs

In [68]:
i = interactive(count_first_digit, var=list(pop.columns))
display(i)
obs = i.result

interactive(children=(Dropdown(description='var', options=('POPESTIMATE2020', 'POPESTIMATE2021', 'POPESTIMATE2…